In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import os
import json
import fnmatch

In [2]:
# Files path 
general=os.path.join("Output","General")
category=os.path.join("Output", "Category")
# Leagues
leagues=['MLB', 'MLS', 'NBA', 'NFL', 'NHL']

In [3]:
# Example analysis for NFL, from "General" data
nfl_list=os.listdir(os.path.join(general, "NFL"))

In [4]:
nfl_list

['Arizona_Cardinals_restaurants.json',
 'Atlanta_Falcons_restaurants.json',
 'Baltimore_Ravens_restaurants.json',
 'Buffalo_Bills_restaurants.json',
 'Carolina_Panthers_restaurants.json',
 'Chicago_Bears_restaurants.json',
 'Cincinnati_Bengals_restaurants.json',
 'Cleveland_Browns_restaurants.json',
 'Dallas_Cowboys_restaurants.json',
 'Denver_Broncos_restaurants.json',
 'Detroit_Lions_restaurants.json',
 'Green_Bay_Packers_restaurants.json',
 'Houston_Texans_restaurants.json',
 'Indianapolis_Colts_restaurants.json',
 'Jacksonville_Jaguars_restaurants.json',
 'Kansas_City_Chiefs_restaurants.json',
 'Las_Vegas_Raiders_restaurants.json',
 'Los_Angeles_Chargers_restaurants.json',
 'Los_Angeles_Rams_restaurants.json',
 'Miami_Dolphins_restaurants.json',
 'Minnesota_Vikings_restaurants.json',
 'New_England_Patriots_restaurants.json',
 'New_Orleans_Saints_restaurants.json',
 'New_York_Giants_restaurants.json',
 'New_York_Jets_restaurants.json',
 'Philadelphia_Eagles_restaurants.json',
 'Pitt

In [5]:
# drop teams that play in the same stadium/duplicated cities
def remove_duplicated_city(inputList, dupList):
    drop_list=[]
    lista=inputList.copy()
    for v in lista:
        for d in dupList:
            if fnmatch.fnmatch(v, f'{d}*.json'):
                drop_list.append(v)
    for t in drop_list[1::2]:
        lista.remove(t)
    return lista    

In [6]:
new_list=remove_duplicated_city(nfl_list, ['New_York', "Los_Angeles"])

In [7]:
new_list

['Arizona_Cardinals_restaurants.json',
 'Atlanta_Falcons_restaurants.json',
 'Baltimore_Ravens_restaurants.json',
 'Buffalo_Bills_restaurants.json',
 'Carolina_Panthers_restaurants.json',
 'Chicago_Bears_restaurants.json',
 'Cincinnati_Bengals_restaurants.json',
 'Cleveland_Browns_restaurants.json',
 'Dallas_Cowboys_restaurants.json',
 'Denver_Broncos_restaurants.json',
 'Detroit_Lions_restaurants.json',
 'Green_Bay_Packers_restaurants.json',
 'Houston_Texans_restaurants.json',
 'Indianapolis_Colts_restaurants.json',
 'Jacksonville_Jaguars_restaurants.json',
 'Kansas_City_Chiefs_restaurants.json',
 'Las_Vegas_Raiders_restaurants.json',
 'Los_Angeles_Chargers_restaurants.json',
 'Miami_Dolphins_restaurants.json',
 'Minnesota_Vikings_restaurants.json',
 'New_England_Patriots_restaurants.json',
 'New_Orleans_Saints_restaurants.json',
 'New_York_Giants_restaurants.json',
 'Philadelphia_Eagles_restaurants.json',
 'Pittsburgh_Steelers_restaurants.json',
 'San_Francisco_49ers_restaurants.json

## Let's have a look to the data in one file

In [61]:
ratings=[]
review_count=[]
price=[]
for i in range(jsonData['total']):
    try:
        ratings.append(jsonData['businesses'][i]['rating'])
        review_count.append(jsonData['businesses'][i]['review_count'])
        price.append(jsonData['businesses'][i]['price'])
    except:
        ratings.append(np.nan)
        review_count.append(np.nan)
        price.append(np.nan)


In [10]:
#function to drop nan from a list
def drop_nan(_list):
    return [_ for _ in _list if _ is not np.nan]

In [11]:
# map the price list to numeric values
def pricemap(alist):
    price_map= {"$": 1, "$$": 2, "$$$": 3, "$$$$": 4}
    for k, v in enumerate(alist):
        if v in price_map:
            alist[k]=price_map[v]

In [76]:
def price_mapper(dictio):
    price_map= {"$": 1, "$$": 2, "$$$": 3, "$$$$": 4}
    for i in range(dictio['total']):
        if 'price' in dictio['businesses'][i] and dictio['businesses'][i]['price'] in price_map:
            dictio['businesses'][i]['price']=price_map[dictio['businesses'][i]['price']]
        else:
            dictio['businesses'][i]['price']=np.nan

In [77]:
def openfile(filep):
    with open(filep, encoding="utf-8") as f:
        jsonData=json.load(f)
    return jsonData

In [82]:
NFL_dict={}
for file in new_list:
    business_price=[]
    business_rating=[]
    business_reviews=[]
    file_path=os.path.join(general, 'NFL', file)
    fileData=openfile(file_path)
    price_mapper(fileData)
    abs_total=fileData['absolute total']
    total=fileData['total']
    for t in range(fileData['total']):
        business_price.append(fileData['businesses'][t]['price'])
        try:
            business_rating.append(fileData['businesses'][t]['rating'])
            business_reviews.append(fileData['businesses'][t]['review_count'])
        except:
            business_rating.append(np.nan)
            business_reviews.append(np.nan)    
    
    bprice=drop_nan(business_price)
    brating=drop_nan(business_rating)
    breview=drop_nan(business_reviews)
    NFL_dict[fileData['team']]=(abs_total, np.mean(bprice), np.mean(brating), np.mean(breview))

In [84]:
statistics_df = pd.DataFrame.from_dict(NFL_dict, orient = "index",
                           columns = ("Number of businesses", "Average Price (1-4)", "Average Ratings (1-5)", "Average Number of Reviews"))

In [86]:
statistics_df

,Number of businesses,Average Price (1-4),Average Ratings (1-5),Average Number of Reviews
Arizona Cardinals,101,1.466667,3.089109,141.544554
Atlanta Falcons,665,1.566810,3.386466,172.425564
Baltimore Ravens,784,1.718095,3.611607,124.137755
Buffalo Bills,64,1.553191,3.281250,43.609375
Carolina Panthers,448,1.680000,3.685268,158.205357
Chicago Bears,1200,1.729040,3.682500,269.505000
Cincinnati Bengals,425,1.605096,3.808235,122.908235
Cleveland Browns,376,1.714815,3.728723,116.529255
Dallas Cowboys,223,1.406977,3.275785,105.493274
Denver Broncos,715,1.706542,3.881818,239.097902
